# Introduction

In [1]:
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd


print('TensorFlow: ', tf.__version__)
print('OpenCV: ', cv2.__version__)

## Explore the Dataset

In [4]:
dataset_path = '../input/damaged-cars-dataset/damage_car_dataset'

categories = os.listdir(dataset_path)

print('Total Number of categories: ', len(categories))
print('Names of categories are following: \n')
for cat in categories:
    dir_name = os.path.join(dataset_path, cat)
    print(cat, ' : ', len(os.listdir(dir_name)), 'images')

## Data Preprocessing

In [11]:
image_size = (224, 224)
batch_size = 32

# Can't understand this function.
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '../input/damaged-cars-dataset/damage_car_dataset',
    validation_split=0.2,
    subset='training',
    seed=1337,
    image_size=image_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '../input/damaged-cars-dataset/damage_car_dataset',
    validation_split=0.2,
    subset='validation',
    seed=1337,
    image_size=image_size,
    batch_size=batch_size
)

In [12]:
# Configure Dataset for performance

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Create the model

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3,)),
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(8)
])

model.summary()

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
)

## Train Model

In [13]:
history_model =  model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    verbose=0
)

## Save the model

In [15]:
model.save('Car_Damange_Assesment_Baseline_Model.h5')

## Plot the results

In [18]:
history_model.history.keys()

In [28]:
loss = history_model.history['loss']
accuracy = history_model.history['accuracy']
val_loss = history_model.history['val_loss']
val_accuracy = history_model.history['val_accuracy']

epochs = range(1, len(accuracy) + 1 )

plt.plot(epochs, loss, 'ro', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.savefig('loss.png')

plt.figure()
plt.plot(epochs, accuracy, 'bo', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.savefig('accuracy.png')

plt.show()